<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       XGBoost and XGBoostPredict Functions
       in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>XGBoost function(eXtreme Gradient Boosting) is an implementation of the gradient boosted decision tree designed for speed and performance. In gradient boosting, each iteration fits a model to the residuals (errors) of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models. XGBoostPredict function runs the predictive algorithm based on the model generated by XGBoost. <br>In this notebook we will see how we can use the XGBoost and XGBoostPredict functions available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_XGBoost_and_XGBoostPredict_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_local');" 

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_TitanicSurvival", "Passenger_Data"))
print("Shape of the data: ", tdf.shape)
tdf

<p style = 'font-size:16px;font-family:Arial'>First let us divide our dataset into train and test datasets for our model creation. </p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(data = tdf,
                                    id_column="passenger",
                                    train_size=0.80,
                                    test_size=0.20,
                                    seed=42)
       
dataset_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
dataset_test  = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

<p style = 'font-size:16px;font-family:Arial'>Now we will use XGBoost alogrithm on our train dataset. Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(XGBoost)

In [ ]:
XGBoost_out = XGBoost(data=dataset_train,
                        input_columns=["age", "survived", "pclass"],
                        response_column = 'fare',
                        max_depth=3,
                        lambda1 = 1000.0,
                        model_type='Regression',
                        seed=-1,
                        shrinkage_factor=0.1,
                        iter_num=2)

In [ ]:
XGBoost_out.result

In [ ]:
XGBoost_out.output_data

<p style = 'font-size:16px;font-family:Arial'>Now we will apply XGBoostPredict function for the predictions. Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(XGBoostPredict)

In [ ]:
XGBoostPredict_out = XGBoostPredict(newdata=dataset_test,
                                      object=XGBoost_out.result,
                                      id_column='passenger',
                                      object_order_column=['task_index', 'tree_num',
                                                           'iter', 'tree_order'])

# Print the result DataFrame.
XGBoostPredict_out.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_TitanicSurvival');"     # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>XGBoost function reference: <a href = 'https://docs.teradata.com/search/all?query=XGBoost&content-lang=en-US'>here</a></li>
    <li>XGBoostPredict function reference: <a href = 'https://docs.teradata.com/search/all?query=XGBoostPredict&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>